In [2]:
import numpy as np
import random
import sklearn

In [3]:
a = [
    [[1, 2, 1], [1, 2, 1]],
    [[1, 2, 1], [0, 0, 0]]
]
b = [
    [[1, 2, 1], [1, 2, 1]],
    [[0, 0, 0], [1, 2, 1]]
]

print(np.add(a, b))
print(np.subtract(a, b))

[[[2 4 2]
  [2 4 2]]

 [[1 2 1]
  [1 2 1]]]
[[[ 0  0  0]
  [ 0  0  0]]

 [[ 1  2  1]
  [-1 -2 -1]]]


In [4]:
class NN:
    def __init__(self, sizes):
        # suppose sizes = [1, 2, 3]
        # then input layer has 1 neuron, hidden 2, and output 3 neurons
        self.sizes = sizes
        self.num_layers = len(sizes)
        
        # an array containing the weights matrix for every
        # layer excluding the input layer
        self.weights = []
        for i in range(1, self.num_layers):
            self.weights.append(np.random.randn(sizes[i], sizes[i - 1]))
            
        # an array containing the bias matrices for every layer
        # except the input layer
        self.biases = [np.random.randn(sz, 1) for sz in sizes[1:]]
    
    # a function which will calc our output from weights, biases and input
    def feed_forward(self, a):
        for w, b in zip(self.weights, self.biases):
            a = self.sigmoid(np.dot(w, a) + b)
        return a
    def predict(self, x, single = True):
        if single:
            return np.argmax(self.feed_forward(x))
        else:
            res = []
            for xi in x:
                res.append(self.predict(xi))
            return res
    
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))
    
    def sigmoid_prime(self, z):
        return self.sigmoid(z) * (1.0 - self.sigmoid(z))
    
    # function to calc accuracy (for classification only)
    # format = 
    def evaluate(self, data, labels):
        output = [np.argmax(self.feed_forward(x)) for x in data]
        labels = [np.argmax(li) for li in labels]
        acc = 0
        for y_hat, y in zip(output, labels):
            acc = acc + int(y_hat == y)
            
        return (acc * 100) / len(data)
    
    
    # a function which create batches for our SGD to work
    def get_batches(self, data, labels, batch_size):
        n = len(data)
        data, labels = sklearn.utils.shuffle(data, labels)
        batches = [
            data[i:i + batch_size] for i in range(0, n, batch_size)
        ]
        batches_labels = [
            labels[i:i + batch_size] for i in range(0, n, batch_size)
        ]
        return batches, batches_labels
    
    # a function for calling schotastic gradient descent on our data
    def SGD(self, data, labels, lr = 0.01, epochs = 10, batch_size = 3, test_data = None, test_labels = None, test = False):
        n_test = 0
        if test:
            n_test = len(test_data)
        n = len(data)
        
        # main algorithm
        for epoch in range(epochs):
            batches_data, batches_labels = self.get_batches(data, labels, batch_size)
            # updating parameters considering every batch
            for batch, label in zip(batches_data, batches_labels):
                self.update_parameters(batch, label, lr)
            
            if test:
                print ("Epoch:", epoch, "accuracy:", self.evaluate(test_data, test_labels))
            else:
                print("Epoch:", epoch, "complete")
    
    def get_next(self, w, activation, b):
        z = np.dot(w, activation) + b
        return z, self.sigmoid(z)
    
    # BP1
    def get_del_L(self, a_L, y, z_L):
        return (a_L - y) * self.sigmoid_prime(z_L)
    
    # BP2
    def get_del_l(self, w_next, del_next, z_l):
        return np.dot(w_next.transpose(), del_next) * self.sigmoid_prime(z_l)
    
    # BP3
    def get_del_b(self, del_l):
        return del_l
    
    # BP4
    def get_del_w(self, del_l, a_prev):
        return np.dot(del_l, a_prev.transpose())
    
    
    def backpropagation(self, X, y):
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        
        activation = X
        activations = [X]
        zs = []
        
        for w, b in zip(self.weights, self.biases):
            z, activation = self.get_next(w, activation, b)
            zs.append(z)
            activations.append(activation)
            
        del_L = self.get_del_L(activations[-1], y, zs[-1])
        
        grad_b[-1] = self.get_del_b(del_L)
        grad_w[-1] = self.get_del_w(del_L, activations[-2])
        
        # layers are 0, 1, 2, ..., num_layers - 1
        # num_layers - 2 ==> 1
        del_l = del_L
        for l in range(self.num_layers - 2, 0):
            del_l = self.get_del_l(self.weights[l + 1], del_l, zs[l])
            grad_b[l] = self.get_del_b(del_l)
            grad_w[l] = self.get_del_w(del_l, activations[l - 1])
        
        return (grad_w, grad_b)
        
        
        
    def update_parameters(self, batch, label, lr):
        grad_w = [np.zeros(w.shape) for w in self.weights]
        grad_b = [np.zeros(b.shape) for b in self.biases]
        
        for i in range(len(batch)):
            del_w, del_b = self.backpropagation(batch[i], label[i])
            grad_w = np.add(grad_w, del_w)
            grad_b = np.add(grad_b, del_b)
        
        self.weights = np.subtract(self.weights, lr * grad_w / len(batch))
        self.biases = np.subtract(self.biases, lr * grad_b / len(batch))
            

In [5]:
myNN = NN([4, 20, 3])

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets

In [7]:
iris = datasets.load_iris()
print(iris.keys())
print(iris.target_names)
print(iris.feature_names)

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])
['setosa' 'versicolor' 'virginica']
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [8]:
def preprocessing(dataset):
    DS = dataset.data
    scaler = MinMaxScaler()
    DS = scaler.fit_transform(DS)
    y = []
    yo = [
        [[1], [0], [0]], 
        [[0], [1], [0]], 
        [[0], [0], [1]]
    ]
    X = []
    nf = len(DS[0])
    for i in range(len(DS)):
        X.append(np.resize(DS[i], (nf, 1)))
    for yi in dataset.target:
        y.append(yo[yi])
    return np.array(X), np.array(y)

In [9]:
X, y = preprocessing(iris)
print(X.shape)
print(y.shape)

(150, 4, 1)
(150, 3, 1)


In [10]:
myNN.SGD(X, y, 0.1, 5000, 50, X, y, True)

Epoch: 0 accuracy: 33.333333333333336
Epoch: 1 accuracy: 33.333333333333336
Epoch: 2 accuracy: 33.333333333333336
Epoch: 3 accuracy: 33.333333333333336
Epoch: 4 accuracy: 33.333333333333336
Epoch: 5 accuracy: 33.333333333333336
Epoch: 6 accuracy: 33.333333333333336
Epoch: 7 accuracy: 33.333333333333336
Epoch: 8 accuracy: 33.333333333333336
Epoch: 9 accuracy: 33.333333333333336
Epoch: 10 accuracy: 33.333333333333336
Epoch: 11 accuracy: 36.0
Epoch: 12 accuracy: 46.0
Epoch: 13 accuracy: 56.0
Epoch: 14 accuracy: 64.0
Epoch: 15 accuracy: 65.33333333333333
Epoch: 16 accuracy: 66.0
Epoch: 17 accuracy: 66.0
Epoch: 18 accuracy: 66.0
Epoch: 19 accuracy: 66.66666666666667
Epoch: 20 accuracy: 66.66666666666667
Epoch: 21 accuracy: 66.66666666666667
Epoch: 22 accuracy: 66.66666666666667
Epoch: 23 accuracy: 66.66666666666667
Epoch: 24 accuracy: 66.0
Epoch: 25 accuracy: 66.66666666666667
Epoch: 26 accuracy: 66.0
Epoch: 27 accuracy: 66.0
Epoch: 28 accuracy: 66.0
Epoch: 29 accuracy: 66.0
Epoch: 30 accur

Epoch: 245 accuracy: 77.33333333333333
Epoch: 246 accuracy: 78.0
Epoch: 247 accuracy: 77.33333333333333
Epoch: 248 accuracy: 77.33333333333333
Epoch: 249 accuracy: 77.33333333333333
Epoch: 250 accuracy: 77.33333333333333
Epoch: 251 accuracy: 77.33333333333333
Epoch: 252 accuracy: 77.33333333333333
Epoch: 253 accuracy: 77.33333333333333
Epoch: 254 accuracy: 77.33333333333333
Epoch: 255 accuracy: 78.0
Epoch: 256 accuracy: 78.0
Epoch: 257 accuracy: 78.0
Epoch: 258 accuracy: 78.0
Epoch: 259 accuracy: 78.0
Epoch: 260 accuracy: 78.0
Epoch: 261 accuracy: 77.33333333333333
Epoch: 262 accuracy: 77.33333333333333
Epoch: 263 accuracy: 77.33333333333333
Epoch: 264 accuracy: 77.33333333333333
Epoch: 265 accuracy: 78.0
Epoch: 266 accuracy: 77.33333333333333
Epoch: 267 accuracy: 77.33333333333333
Epoch: 268 accuracy: 78.0
Epoch: 269 accuracy: 78.0
Epoch: 270 accuracy: 78.0
Epoch: 271 accuracy: 78.0
Epoch: 272 accuracy: 78.0
Epoch: 273 accuracy: 78.0
Epoch: 274 accuracy: 78.0
Epoch: 275 accuracy: 78.0

KeyboardInterrupt: 

In [11]:
for w, b in zip(myNN.weights, myNN.biases):
    print('w', w.shape)
    print('b', b.shape)

w (20, 4)
b (20, 1)
w (3, 20)
b (3, 1)


In [12]:
res = myNN.predict(X, False)

In [13]:
print(sum(res == iris.target) / len(res))

0.78


In [14]:
print(res)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [15]:
# next test it on hand writting recognition and then make video

In [16]:
from sklearn import datasets

In [17]:
digits = datasets.load_digits()
print(digits.data.shape)
print(digits.target)

(1797, 64)
[0 1 2 ... 8 9 8]


In [18]:
def preprocessing_mnist(dataset):
    DS = dataset.data
    scaler = MinMaxScaler()
    DS = scaler.fit_transform(DS)
    X = []
    y = []
    nf = len(DS[0])
    num_classes = 10
    yo = [
        np.zeros((num_classes, 1)) for i in range(num_classes)
    ]
    for i in range(num_classes):
        yo[i][i][0] = 1
    # see preprocessing function for looking at the structure of yo
    for i in range(len(DS)):
        #(64,) => (64,1)  #because we need vector
        X.append(np.resize(DS[i], (nf, 1)))
        y.append(yo[dataset.target[i]])
        
    return np.array(X), np.array(y)

In [19]:
X_mnist, y_mnist = preprocessing_mnist(digits)

In [20]:
NN_mnist = NN([64, 256, 128, 64, 32, 10])

In [23]:
NN_mnist.SGD(X_mnist, y_mnist, 
             lr = 0.5, epochs = 100,
             batch_size = 100, 
             test_data = X_mnist, test_labels = y_mnist, test = True)

In [24]:
predictions_mnist = NN_mnist.predict(X_mnist, False)

In [25]:
print(sum(predictions_mnist == digits.target) / len(predictions_mnist) * 100)

23.984418475236506


In [27]:
!pip install --upgrade tensorflow

  Using cached tensorflow-2.8.0-cp37-cp37m-win_amd64.whl (437.9 MB)
     -------------------------------------- 126.7/126.7 KB 1.1 MB/s eta 0:00:00
     -------------------------------------- 462.5/462.5 KB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 KB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 3.4/3.4 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached numpy-1.21.5-cp37-cp37m-win_amd64.whl (14.0 MB)
     ---------------------------------------- 42.6/42.6 KB ? eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 13.9/13.9 MB 1.7 M

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\abhishek sharma\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\~umpy\\.libs\\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\abhishek sharma\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
import tensorflow
from tensorflow.keras.datasets.cifar10 import load_data as cifar

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar()

In [ ]:
def preprocessing_mnist(dataset):
    DS = dataset.data
    scaler = MinMaxScaler()
    DS = scaler.fit_transform(DS)
    X = []
    y = []
    nf = len(DS[0])    
    num_classes = 10
    yo = [
        np.zeros((num_classes, 1)) for i in range(num_classes)
    ]
    for i in range(num_classes):
        yo[i][i][0] = 1
    # see preprocessing function for looking at the structure of yo
    for i in range(len(DS)):
        #(64,) => (64,1)  #because we need vector
        X.append(np.resize(DS[i], (nf, 1)))
        y.append(yo[dataset.target[i]])
        
    return np.array(X), np.array(y)

In [ ]:
def preprocessing_cifar10(X_, y_):
    X = []
    y = []
    nf1 = len(X_[0])
    nf2 = len(X_[0][0])
    nf3 = len(X_[0][0][0])
    
    nd = len(X_)
    num_classes = 10
    yo = [np.zeros((num_classes, 1)) for i in range(num_classes)]
    for i in range(num_classes):
        yo[i][i][0] = 1
    
    for i in range(nd):
        X.append(np.resize(X_[i], (nf1 * nf2 * nf3, 1)))
        y.append(yo[y_[i][0]])